## FullAdder - Combinational Circuits

This notebook walks through the implementation of a basic combinational circuit, a full adder. This example introduces many of the features of `Magma` including circuits, wiring, operators, and the type system.

Start by importing `magma` and `mantle`. `magma` is the core system which implements circuits and the methods to compose them, and `mantle` is a library of useful circuits. 

In [1]:
import warnings
with warnings.catch_warnings():  # surpress warnings from other dependencies
    import magma as m
    import mantle

/Users/hanrahan/miniconda3/lib/python3.7/site-packages/pysmt/walkers/generic.py:43: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if len(nodetypes) == 1 and isinstance(nodetypes[0], collections.Iterable):


A full adder has three single bit inputs, and returns the sum and the carry. The sum is the *exclusive or* of the 3 bits, the carry is 1 if any two of the inputs bits are 1. Here is a schematic of a full adder circuit (from `logisim`).

![Full Adder](images/full_adder_logisim.png)

We start by defining a Python function that implements a full adder. 
The full adder function takes three single bit inputs and returns two outputs as a tuple.
The first element of tuple is the sum, the second element is the carry.
We compute the sum and carry using standard Python bitwise operators `&`, `|`, and `^`.

In [2]:
def fulladder(A, B, C):
    return A ^ B ^ C, A & B | B & C | C & A  # sum, carry

We can test our Python function to verify that our implementation behaves as expected. 
We'll use the standard Python `assert` pattern.

In [3]:
assert fulladder(1, 0, 0) == (1, 0), "Failed"
assert fulladder(0, 1, 0) == (1, 0), "Failed"
assert fulladder(1, 1, 0) == (0, 1), "Failed"
assert fulladder(1, 0, 1) == (0, 1), "Failed"
assert fulladder(1, 1, 1) == (1, 1), "Failed"
print("Success!")

Success!


### Circuits

Now that we have an implementation of `fulladder` as a Python function, 
we'll use it to construct a `magma` `Circuit`. 
A `Circuit` in `magma` corresponds to a `module` in `verilog`.

In [4]:
class FullAdder(m.Circuit):
    io = m.IO(I0=m.In(m.Bit),
              I1=m.In(m.Bit),
              CIN=m.In(m.Bit),
              O=m.Out(m.Bit),
              COUT=m.Out(m.Bit))
    
    O, COUT = fulladder(io.I0, io.I1, io.CIN)
    io.O @= O
    io.COUT @= COUT

First, notice that the `FullAdder` is a subclass of `Circuit`. All `magma` circuits are classes in python.

Second, the function `IO` creates the interface to the circuit. 
The arguments to`IO` are keyword arguments. 
The key is the name of the argument in the circuit, and the value is its type. 
In this circuit, all the inputs and outputs have `Magma` type `Bit`. 
We also qualify each type as an input or an output using the functions `In` and `Out`.

Note that when we call the python function `fulladder`
it is passed `magma` values not standard python values.
In the previous cell,  we tested `fulladder` with standard python ints,
while in this case, the values passed to the Python `fulladder` function 
are `magma` values of type `Bit`.
The Python bitwise operators for `Magmz` types are overloaded to automatically create subcircuits to compute logical functions.

`fulladder` returns two values.
These values are assigned to the python variables `O` and `COUT`. 
Remember that assigning to a Python variable 
sets the variable to refer to the object.
`magma` values are Python objects,
so assigning an object to a variable creates a reference to that `magma` value.
In order to complete the definition of the circuit, 
`O` and `COUT` need to be *wired* to the outputs in the interface.

The python `@=` operator is overloaded to perform wiring.

### Combinational Syntax

In the above example, you'll notice that we have to define two interfaces:
1. the interface to the Python `fulladder` function
2. the interface to the `magma` `fulladdr` circuit.

The `fulladder` is an example of a common `magma` pattern where pure functions are used to represent combinational circuits.  To simplify the declaration of these circuits, `magma` provides the `m.circuit.combinational` decorator which can be used to mark pure functions that should be compiled into `magma` circuits.  This decorator allows us to define the interface of the circuit using Python 3's type annotation syntax.  The implementation will inspect the function definition and generate the corresponding magma circuit definition. Here is an example:

In [5]:
@m.circuit.combinational
def fulladder2(A: m.Bit, B: m.Bit, C:m.Bit) -> (m.Bit, m.Bit):
    return A ^ B ^ C, A & B | B & C | C & A  # sum, carry

We can inspect the generated circuit definition by referencing the `circuit_definition` attribute of the function.  We'll see that it has instantiated a set of `and`, `or`, and `xor` primitives to implement the logic, and handled the wiring of the inputs and outputs.

In [6]:
from magma.ir import compile

print(repr(fulladder2.circuit_definition))

fulladder2 = DefineCircuit("fulladder2", "A", In(Bit), "B", In(Bit), "C", In(Bit), "O0", Out(Bit), "O1", Out(Bit))
magma_Bit_and_inst0 = magma_Bit_and()
magma_Bit_and_inst1 = magma_Bit_and()
magma_Bit_and_inst2 = magma_Bit_and()
magma_Bit_or_inst0 = magma_Bit_or()
magma_Bit_or_inst1 = magma_Bit_or()
magma_Bit_xor_inst0 = magma_Bit_xor()
magma_Bit_xor_inst1 = magma_Bit_xor()
wire(fulladder2.A, magma_Bit_and_inst0.in0)
wire(fulladder2.B, magma_Bit_and_inst0.in1)
wire(fulladder2.B, magma_Bit_and_inst1.in0)
wire(fulladder2.C, magma_Bit_and_inst1.in1)
wire(fulladder2.C, magma_Bit_and_inst2.in0)
wire(fulladder2.A, magma_Bit_and_inst2.in1)
wire(magma_Bit_and_inst0.out, magma_Bit_or_inst0.in0)
wire(magma_Bit_and_inst1.out, magma_Bit_or_inst0.in1)
wire(magma_Bit_or_inst0.out, magma_Bit_or_inst1.in0)
wire(magma_Bit_and_inst2.out, magma_Bit_or_inst1.in1)
wire(fulladder2.A, magma_Bit_xor_inst0.in0)
wire(fulladder2.B, magma_Bit_xor_inst0.in1)
wire(magma_Bit_xor_inst0.out, magma_Bit_xor_inst1.in0)
w

We can also inspect the code generated by the decorator by looking in the `.magma` directory for a file named `.magma/fulladder2.py`.  When using `m.circuit.combinational`, `magma` will generate a file matching the name of the decorated function.  You'll notice that the generated code looks very similar to the handwritten code above, except that magma introduces an extra temporary variable (this is an artifact of the SSA pass that `magma` runs to handle `if`/`else` statements)

In [7]:
with open(".magma/fulladder2.py") as f:
    print(f.read())

import magma as m
from mantle import mux as phi


class fulladder2(m.Circuit):
    io = m.IO(A=m.In(m.Bit), B=m.In(m.Bit), C=m.In(m.Bit), O0=m.Out(m.Bit),
        O1=m.Out(m.Bit))
    __magma_ssa_return_value_0 = (io.A ^ io.B ^ io.C, io.A & io.B | io.B &
        io.C | io.C & io.A)
    O0, O1 = __magma_ssa_return_value_0
    m.wire(O0, io.O0)
    m.wire(O1, io.O1)



### Testing using Fault
`fault` is a [python package](https://github.com/leonardt/fault) for testing magma circuits.  By default, `fault` is quiet, so we begin by enabling logging using the built-in `logging` module

In [8]:
import logging
logging.basicConfig(level=logging.INFO)
import fault

`fault` is a staged metaprogramming environment built upon the `Tester` class.  A `Tester` is instantiated with a `magma` circuit

In [9]:
tester = fault.Tester(FullAdder)

An instance of a `Tester` has an attribute `.circuit` that enables the user to record test actions.  For example, inputs to a circuit can be *poked* by setting the attribute corresponding to the input port name.

In [10]:
tester.circuit.I0 = 1
tester.circuit.I1 = 1
tester.circuit.CIN = 1

`fault`'s default `Tester` provides the semantics of a cycle accurate simulator, so, unlike verilog, pokes do not create events that trigger computation.  Instead, these poke values are staged, and the propogation of their effect occurs when the user calls the `eval` action.

In [11]:
tester.eval()

To assert that the output of the circuit is equal to a value, we use the `expect` method that are defined on the attributes corresponding to circuit output ports

In [12]:
tester.circuit.O.expect(1)
tester.circuit.COUT.expect(1)

Because `fault` is a staged programming environment, the above actions are not executed until we have advanced to the next stage.  In the first stage, the user records test actions (e.g. poke, eval, expect).  In the second stage, the test is compiled and run using a target runtime.  Here's examples of running the test using `magma`'s python simulator, the `coreir` c++ simulator, and `verilator`.

In [13]:
tester.compile_and_run("python")

INFO:root:Running tester...
INFO:root:Success!


In [14]:
tester.compile_and_run("coreir")

INFO:root:Running tester...
INFO:root:Success!


In [15]:
# compile_and_run throws an exception if the test fails
tester.compile_and_run("verilator", magma_output="coreir-verilog")

INFO:root:Running tester...
INFO:root:Success!


### Testing using the Python Simulator

Let's test our `FullAdder` circuit by comparing what it computes to the original python function. We do this by running a python circuit simulator and asserting that the values computed by the simulator are the same as the values computed by the python function.

In [16]:
from magma.simulator import PythonSimulator

fulladder_magma = PythonSimulator(FullAdder)

assert fulladder_magma(1, 0, 0) == fulladder(1, 0, 0), "Failed"
assert fulladder_magma(0, 1, 0) == fulladder(0, 1, 0), "Failed"
assert fulladder_magma(1, 1, 0) == fulladder(1, 1, 0), "Failed"
assert fulladder_magma(1, 0, 1) == fulladder(1, 0, 1), "Failed"
assert fulladder_magma(1, 1, 1) == fulladder(1, 1, 1), "Failed"
print("Success!")

Success!


Here is another way to run the simulator.

In [17]:
fulladder_magma.set_value(FullAdder.I0, 1)
fulladder_magma.set_value(FullAdder.I1, 1)
fulladder_magma.set_value(FullAdder.CIN, 1)
fulladder_magma.evaluate()
assert fulladder_magma.get_value(FullAdder.O) == 1, "Failed"
assert fulladder_magma.get_value(FullAdder.COUT) == 1, "Failed"
print("Success!")

Success!


### Generate python

In [18]:
from magma.ir import compile
print(compile(FullAdder))

FullAdder = DefineCircuit("FullAdder", "I0", In(Bit), "I1", In(Bit), "CIN", In(Bit), "O", Out(Bit), "COUT", Out(Bit))
magma_Bit_and_inst0 = magma_Bit_and()
magma_Bit_and_inst1 = magma_Bit_and()
magma_Bit_and_inst2 = magma_Bit_and()
magma_Bit_or_inst0 = magma_Bit_or()
magma_Bit_or_inst1 = magma_Bit_or()
magma_Bit_xor_inst0 = magma_Bit_xor()
magma_Bit_xor_inst1 = magma_Bit_xor()
wire(FullAdder.I0, magma_Bit_and_inst0.in0)
wire(FullAdder.I1, magma_Bit_and_inst0.in1)
wire(FullAdder.I1, magma_Bit_and_inst1.in0)
wire(FullAdder.CIN, magma_Bit_and_inst1.in1)
wire(FullAdder.CIN, magma_Bit_and_inst2.in0)
wire(FullAdder.I0, magma_Bit_and_inst2.in1)
wire(magma_Bit_and_inst0.out, magma_Bit_or_inst0.in0)
wire(magma_Bit_and_inst1.out, magma_Bit_or_inst0.in1)
wire(magma_Bit_or_inst0.out, magma_Bit_or_inst1.in0)
wire(magma_Bit_and_inst2.out, magma_Bit_or_inst1.in1)
wire(FullAdder.I0, magma_Bit_xor_inst0.in0)
wire(FullAdder.I1, magma_Bit_xor_inst0.in1)
wire(magma_Bit_xor_inst0.out, magma_Bit_xor_inst1.i

### Generate Verilog

In [19]:
m.compile("build/FullAdder", FullAdder, output="verilog")
%cat build/FullAdder.v

module FullAdder (input  I0, input  I1, input  CIN, output  O, output  COUT);
wire  magma_Bit_xor_inst0_out;
wire  magma_Bit_xor_inst1_out;
wire  magma_Bit_and_inst0_out;
wire  magma_Bit_and_inst1_out;
wire  magma_Bit_or_inst0_out;
wire  magma_Bit_and_inst2_out;
wire  magma_Bit_or_inst1_out;
magma_Bit_xor magma_Bit_xor_inst0 (.I0(I0), .I1(I1), .O(magma_Bit_xor_inst0_out));
magma_Bit_xor magma_Bit_xor_inst1 (.I0(magma_Bit_xor_inst0_out), .I1(CIN), .O(magma_Bit_xor_inst1_out));
magma_Bit_and magma_Bit_and_inst0 (.I0(I0), .I1(I1), .O(magma_Bit_and_inst0_out));
magma_Bit_and magma_Bit_and_inst1 (.I0(I1), .I1(CIN), .O(magma_Bit_and_inst1_out));
magma_Bit_or magma_Bit_or_inst0 (.I0(magma_Bit_and_inst0_out), .I1(magma_Bit_and_inst1_out), .O(magma_Bit_or_inst0_out));
magma_Bit_and magma_Bit_and_inst2 (.I0(CIN), .I1(I0), .O(magma_Bit_and_inst2_out));
magma_Bit_or magma_Bit_or_inst1 (.I0(magma_Bit_or_inst0_out), .I1(magma_Bit_and_inst2_out), .O(magma_Bit_or_inst1_out));
assign O = magma_Bit_xor_

Generate verilog with `coreir` modules included.

In [20]:
m.compile("build/FullAdder", FullAdder, output="coreir-verilog")
%cat build/FullAdder.v

module corebit_xor (
    input in0,
    input in1,
    output out
);
  assign out = in0 ^ in1;
endmodule

module corebit_or (
    input in0,
    input in1,
    output out
);
  assign out = in0 | in1;
endmodule

module corebit_and (
    input in0,
    input in1,
    output out
);
  assign out = in0 & in1;
endmodule

module FullAdder (
    input I0,
    input I1,
    input CIN,
    output O,
    output COUT
);
wire magma_Bit_and_inst0_out;
wire magma_Bit_and_inst1_out;
wire magma_Bit_and_inst2_out;
wire magma_Bit_or_inst0_out;
wire magma_Bit_or_inst1_out;
wire magma_Bit_xor_inst0_out;
wire magma_Bit_xor_inst1_out;
corebit_and magma_Bit_and_inst0 (
    .in0(I0),
    .in1(I1),
    .out(magma_Bit_and_inst0_out)
);
corebit_and magma_Bit_and_inst1 (
    .in0(I1),
    .in1(CIN),
    .out(magma_Bit_and_inst1_out)
);
corebit_and magma_Bit_and_inst2 (
    .in0(CIN),
    .in1(I0),
    .out(magma_Bit_and_inst2_out)
);
corebit_or magma_Bit_or_inst0 (
    .in0(magma_Bit_and_inst0_out),
    .in1(magma

### Generate CoreIR

In [21]:
m.compile("build/FullAdder", FullAdder, output="coreir")
%cat build/FullAdder.json

{"top":"global.FullAdder",
"namespaces":{
  "global":{
    "modules":{
      "FullAdder":{
        "type":["Record",[
          ["I0","BitIn"],
          ["I1","BitIn"],
          ["CIN","BitIn"],
          ["O","Bit"],
          ["COUT","Bit"]
        ]],
        "instances":{
          "magma_Bit_and_inst0":{
            "modref":"corebit.and"
          },
          "magma_Bit_and_inst1":{
            "modref":"corebit.and"
          },
          "magma_Bit_and_inst2":{
            "modref":"corebit.and"
          },
          "magma_Bit_or_inst0":{
            "modref":"corebit.or"
          },
          "magma_Bit_or_inst1":{
            "modref":"corebit.or"
          },
          "magma_Bit_xor_inst0":{
            "modref":"corebit.xor"
          },
          "magma_Bit_xor_inst1":{
            "modref":"corebit.xor"
          }
        },
        "connections":[
          ["self.I0","magma_Bit_and_inst0.in0"],
          ["self.I1","magma_Bit_and_inst0.in1"],
          ["magma_B

In [22]:
!coreir -i build/FullAdder.json -p instancecount

An instance count of all the primitives
FullAdder | instances in current | instances in children | 
  corebit_and | 3 | 0
  corebit_or | 2 | 0
  corebit_xor | 2 | 0

{"top":"global.FullAdder",
"namespaces":{
  "global":{
    "modules":{
      "FullAdder":{
        "type":["Record",[
          ["I0","BitIn"],
          ["I1","BitIn"],
          ["CIN","BitIn"],
          ["O","Bit"],
          ["COUT","Bit"]
        ]],
        "instances":{
          "magma_Bit_and_inst0":{
            "modref":"corebit.and"
          },
          "magma_Bit_and_inst1":{
            "modref":"corebit.and"
          },
          "magma_Bit_and_inst2":{
            "modref":"corebit.and"
          },
          "magma_Bit_or_inst0":{
            "modref":"corebit.or"
          },
          "magma_Bit_or_inst1":{
            "modref":"corebit.or"
          },
          "magma_Bit_xor_inst0":{
            "modref":"corebit.xor"
          },
          "magma_Bit_xor_inst1":{
            "modref":"corebit.xor